# Sample Reference GOES NC File read with Python Xarray Engine

In [1]:
import json
import boto3
import boto
import fsspec
import fsspec.utils
import numpy as np
import xarray as xr
import zarr
import os
import ujson
import s3fs
import numcodecs

import kerchunk.combine
from kerchunk.zarr import single_zarr
from kerchunk.combine import MultiZarrToZarr
from kerchunk.hdf import SingleHdf5ToZarr
from pathlib import Path

In [3]:
# load meta data into python dict
json_path = "/Users/katrinasharonin/Downloads/kerchunkC/jsons/2023-006.json"
f = open(json_path)
meta = json.load(f)

print('metadata display')
print(meta.keys())

from itertools import islice

def take(n, iterable):
    """Return the first n items of the iterable as a list."""
    return list(islice(iterable, n))

n_items = take(10, meta['refs'].items())
print('sample dict values')
print(n_items[0])
print(n_items[1])
print(n_items[2])
print(n_items[3])
print(n_items[4])
print(n_items[5])

metadata display
dict_keys(['version', 'refs'])
sample dict values
('.zgroup', '{"zarr_format":2}')
('bandn/.zarray', '{\n    "chunks": [\n        1\n    ],\n    "compressor": null,\n    "dtype": "<i4",\n    "fill_value": null,\n    "filters": null,\n    "order": "C",\n    "shape": [\n        1\n    ],\n    "zarr_format": 2\n}')
('bandn/0', '\x01\x00\x00\x00')
('bandn/.zattrs', '{\n    "_ARRAY_DIMENSIONS": [\n        "bandn"\n    ]\n}')
('.zattrs', '{"Conventions":"CF-1.7","LUT_Filenames":"SpaceLookParams(FM2A_CDRL79RevP_PR_09_08_04)-724222106.0.h5 QTableBand01(FM2A_CDRL79RevM_DO_08_00_00)-601637632.0.h5 CalTargetTimeIntervals(FM2A_CDRL79RevP_DO_09_01_00)-611906700.0.h5 BandSaturationLimits(FM2A_CDRL79RevC_DO_07_00_00)-524728502.0.h5 SolarSpaceLookParams(FM2A_CDRL79RevN_DO_09_00_00)-600765435.0.h5 DeadRowListParams(FM2A_CDRL79RevC_DO_07_00_00)-524728502.0.h5 Mirror_Record(FM2A_CDRL79RevC_DO_08_00_00)-604635792.0.h5 KalmanAstroConsts(FM2A_CDRL79RevC_DO_07_00_00)-582860861.0.xml KalmanFi

In [14]:
ds = xr.open_dataset("reference://", engine="zarr", backend_kwargs={
                    "consolidated": False,
                    "storage_options": {"fo": json_path}
                    })

# print(ds.isel(band=slice(0,1),time=slice(0,1),x=slice(0,226),y=slice(0,226))["Rad"].values)
# aim for 17th index for vals
print(ds.isel(band=slice(0, 1), time=slice(0, 1), x=slice(0, 226), y=slice(3842, 4068))["Rad"])

# subset = ds.isel(band=slice(0, 1), time=slice(0, 1), x=slice(0, 226), y=slice(0, 226))["Rad"]
subset = ds.isel(band=slice(0, 1), time=slice(0, 1), x=slice(0, 226), y=slice(3842, 4068))["Rad"]

# Retrieve the values as a NumPy array
values = subset.values

print(values)

# Filter and print non-NaN values
non_nan_values = values[~np.isnan(values)]
print(non_nan_values)

<xarray.DataArray 'Rad' (bandn: 1, time: 1, y: 226, x: 226)>
[51076 values with dtype=float32]
Coordinates:
  * bandn    (bandn) int32 1
    t        (bandn, time) datetime64[ns] ...
  * time     (time) datetime64[us] 2023-01-06T05:00:00
  * x        (x) float64 -0.1519 -0.1518 -0.1518 ... -0.1456 -0.1456 -0.1456
    x_image  (bandn, time) float32 ...
  * y        (y) float64 0.04428 0.04425 0.04423 ... 0.03804 0.03801 0.03798
    y_image  (bandn, time) float32 ...
Attributes:
    ancillary_variables:    DQF
    cell_methods:           t: point area: point
    grid_mapping:           goes_imager_projection
    long_name:              ABI L1b Radiances
    resolution:             y: 0.000028 rad x: 0.000028 rad
    sensor_band_bit_depth:  10
    standard_name:          toa_outgoing_radiance_per_unit_wavelength
    units:                  W m-2 sr-1 um-1
    valid_range:            [0, 1022]
[[[[      nan       nan       nan ...       nan       nan       nan]
   [      nan       nan     

Unfinished; work on local read configuration